In [122]:
import pandas as pd
import numpy as np
import ast
import re
import json

In [219]:
tree_path = '../auspice/flu_seasonal_h3n2_ha_12y_hi_tree.json'
with open(tree_path, 'r') as tree_json:
    tree = json.load(tree_json)

In [305]:
leaf_paths = {}
clock_lengths = {}

def find_path(branch):
    
    node_path.append(branch['clade'])
    if branch['clade'] not in clock_lengths.keys():
        clock_lengths[str(branch['clade'])] = branch['attr']['clock_length']
    if 'children' in branch.keys():
        for child in branch['children']:
            find_path(child)
            node_path.remove(child['clade'])


    elif 'children' not in branch.keys():

        node_path_list = [str(node) for node in node_path]
        leaf_paths[branch['strain']] = [branch['attr']['clade_membership'], node_path_list]

node_path = []
find_path(tree)

df = pd.DataFrame(leaf_paths).T
df.reset_index(inplace=True)
df.columns = ['strain', 'clade_membership', 'path']
df['clade'] = 'unassigned'


In [306]:
#Find clades
max_path_length=df['path'].map(len).max()

current_clade = 0

for internal_branch in reversed(range(0,max_path_length)):
    exclude_assigned = df[df['clade']=='unassigned']
    sub_df = exclude_assigned[exclude_assigned['path'].map(len) > internal_branch]

    path_group= sub_df.groupby((sub_df.path.apply(lambda col: col[0:(internal_branch+1)])).map(tuple))
    for k, v in path_group:
        if len(v)>=100:
            current_clade+=1
            df.at[v.index, 'clade']=current_clade
            
        elif len(v)>=50:
            if clock_lengths[str(k[-1])] >= 0.0008:
                current_clade+=1
                df.at[v.index, 'clade']=current_clade
            
        elif len(v)>=20:
            if clock_lengths[str(k[-1])] >= 0.003:
                current_clade+=1
                df.at[v.index, 'clade']=current_clade

In [307]:
df = df.set_index('strain')

In [308]:
df.groupby(['clade', 'clade_membership']).size()

clade       clade_membership
1           A1b/131K             40
2           A2/re               103
3           A1b/135N             53
4           A1b/135K            119
5           A3                   75
6           A2                   68
            A2/re                26
7           A1                   82
            A1a                   4
            A1b                  31
8           3c2.A               119
9           A1                   67
10          3c3.A                44
11          3c2.A               110
12          3c3                  18
            3c3.B                35
13          A4                   29
14          3c3.A               110
15          3c2.A                88
16          3c3                  89
17          3c3                 100
18          3c2                  81
19          3c                   39
            3c2                  21
            3c3                  41
            3c3.A                21
20          3c                  112

In [270]:
def add_clades(branch, clade_assignments):
    if 'children' in branch.keys():
        for child in branch['children']:
            add_clades(child, df)

    elif 'children' not in branch.keys():
        assigned_clade = df.loc[branch['strain']]['clade']
        branch['attr']['kk_clade'] = assigned_clade

add_clades(tree, df)

with open(tree_path, 'w') as outfile:
    json.dump(tree, outfile, indent=2, sort_keys = True)
